***GENERATED CODE FOR 3110sqlsink PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        #hdfs = HDFileSystem(host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def calculateFormula(df, functionsData, applyOn):
    dfcp = df
    for functionData in functionsData:
        if functionData['useNext']:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
            else:
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['operand1'], functionData['operand2'])
        else:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData['operator'],
                                                            functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                dfcp.drop('dataPrepColumn_Inprocess', axis=1)
            else:
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
    return dfcp


def calculateEquation(df, operator, columnA, columnB):
    if (operator == "add"):
        return df[columnA].add(df[columnB])
    elif (operator == "sub"):
        return df[columnA].sub(df[columnB])
    elif (operator == "mul"):
        return df[columnA].mul(df[columnB])
    elif (operator == "div"):
        return df[columnA].div(df[columnB])
    elif (operator == "mod"):
        return df[columnA].mod(df[columnB])


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run 3110sqlsinkHooks.ipynb
try:
	#sourcePreExecutionHook()

	bostonhousingtrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Boston_Housing_Train.csv', 'filename': 'Boston_Housing_Train.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PERFORMING OPERATIONS***

In [ ]:
#%run 3110sqlsinkHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(bostonhousingtrain,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/Boston_Housing_Train.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/Boston_Housing_Train.csv", "data_source": "localfiles", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/6540b908730bd4ed2218ee3c1698740626/0part.csv", "requestRatio": 1.0, "totalRows": 506, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 15, "numberOfRows": 506, "duplicateRowCount": 0, "stats": [{"column": "CRIM", "alias": "CRIM", "generated": 0, "type": "real", "max": 88.9762, "min": 0.00632, "mean": 3.613523557312254, "missing": 0.0, "stddev": 8.6, "outliers": [8.98296, 13.5222, 9.2323, 8.26725, 11.1081, 18.4982, 19.6091, 15.288, 9.82349, 23.6482, 17.8667, 88.9762, 15.8744, 9.18702, 20.0849, 16.8118, 24.3938, 22.5971, 14.3337, 11.5779, 8.64476, 13.3598, 8.71675, 38.3518, 9.91655, 25.0461, 14.2362, 9.59571, 24.8017, 41.5292, 67.9208, 20.7162, 11.9511, 14.4383, 51.1358, 14.0507, 18.811, 28.6558, 45.7461, 18.0846, 10.8342, 25.9406, 73.5341, 11.8123, 11.0874, 12.0482, 8.79212, 15.8603, 12.2472, 37.6619, 9.33889, 8.49213, 10.0623, 13.9134, 11.1604, 14.4208, 15.1772, 13.6781, 9.39063, 22.0511, 9.72418, 9.96654, 12.8023, 10.6718, 9.92485, 9.32909, 8.24809, 9.51363, 8.20058, 15.5757, 13.0751, 15.0234, 10.233, 14.3337], "validation": []}, {"column": "ZN", "alias": "ZN", "generated": 0, "type": "real", "max": 100.0, "min": 0.0, "mean": 11.363636363636363, "missing": 0.0, "stddev": 23.32, "outliers": [75.0, 75.0, 75.0, 90.0, 85.0, 100.0, 80.0, 80.0, 45.0, 45.0, 45.0, 45.0, 45.0, 45.0, 60.0, 60.0, 80.0, 80.0, 80.0, 80.0, 95.0, 95.0, 82.5, 82.5, 95.0, 95.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 80.0, 80.0, 90.0, 40.0, 40.0, 40.0, 40.0, 40.0, 90.0, 90.0, 55.0, 80.0, 52.5, 52.5, 52.5, 80.0, 80.0, 80.0, 70.0, 70.0, 70.0, 34.0, 34.0, 34.0, 33.0, 33.0, 33.0, 33.0, 35.0, 35.0, 35.0, 55.0, 55.0, 85.0, 80.0, 40.0, 40.0, 60.0, 60.0, 90.0, 80.0, 80.0], "validation": []}, {"column": "INDUS", "alias": "INDUS", "generated": 0, "type": "real", "max": 27.74, "min": 0.46, "mean": 11.13677865612648, "missing": 0.0, "stddev": 6.86, "outliers": [], "validation": []}, {"column": "CHAS", "alias": "CHAS", "generated": 0, "type": "real", "max": 1.0, "min": 0.0, "mean": 0.0691699604743083, "missing": 0.0, "stddev": 0.25, "outliers": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "validation": []}, {"column": "NOX", "alias": "NOX", "generated": 0, "type": "real", "max": 0.871, "min": 0.385, "mean": 0.5546950592885376, "missing": 0.0, "stddev": 0.12, "outliers": [0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871, 0.871], "validation": []}, {"column": "RM", "alias": "RM", "generated": 0, "type": "real", "max": 8.78, "min": 3.561, "mean": 6.284634387351779, "missing": 0.0, "stddev": 0.7, "outliers": [8.069, 7.82, 4.903, 4.926, 7.802, 8.375, 7.929, 7.765, 7.831, 7.875, 7.61, 7.853, 8.034, 8.266, 8.725, 8.04, 7.686, 8.337, 8.247, 8.259, 8.704, 8.398, 8.297, 7.691, 7.82, 7.645, 7.923, 8.78, 3.561, 3.863, 4.906, 4.138, 4.368, 4.652, 4.88, 4.138, 4.628, 4.519], "validation": []}, {"column": "AGE", "alias": "AGE", "generated": 0, "type": "real", "max": 100.0, "min": 2.9, "mean": 68.57490118577076, "missing": 0.0, "stddev": 28.15, "outliers": [], "validation": []}, {"column": "DIS", "alias": "DIS", "generated": 0, "type": "real", "max": 12.1265, "min": 1.1296, "mean": 3.795042687747036, "missing": 0.0, "stddev": 2.11, "outliers": [9.1876, 9.2229, 9.2203, 9.2203, 9.0892, 10.7103, 10.7103, 12.1265, 10.5857, 10.5857], "validation": []}, {"column": "RAD", "alias": "RAD", "generated": 0, "type": "real", "max": 24.0, "min": 1.0, "mean": 9.549407114624506, "missing": 0.0, "stddev": 8.71, "outliers": [], "validation": []}, {"column": "TAX", "alias": "TAX", "generated": 0, "type": "real", "max": 711.0, "min": 187.0, "mean": 408.2371541501976, "missing": 0.0, "stddev": 168.54, "outliers": [], "validation": []}, {"column": "PTRATIO", "alias": "PTRATIO", "generated": 0, "type": "real", "max": 22.0, "min": 12.6, "mean": 18.455533596837945, "missing": 0.0, "stddev": 2.16, "outliers": [12.6, 12.6, 12.6, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.6], "validation": []}, {"column": "B", "alias": "B", "generated": 0, "type": "real", "max": 396.9, "min": 0.32, "mean": 356.6740316205534, "missing": 0.0, "stddev": 91.29, "outliers": [288.99, 303.42, 306.38, 232.6, 248.31, 70.8, 344.91, 338.63, 262.76, 172.91, 169.27, 341.6, 343.28, 261.95, 321.02, 88.01, 88.63, 338.92, 240.16, 227.61, 297.09, 330.04, 292.29, 348.13, 348.93, 341.6, 316.03, 131.42, 347.88, 285.83, 338.16, 329.46, 332.09, 314.64, 179.36, 2.6, 35.05, 28.79, 210.97, 88.27, 27.25, 21.57, 127.36, 16.45, 48.45, 318.75, 319.98, 291.55, 2.52, 3.65, 7.68, 24.65, 18.82, 96.73, 60.72, 83.45, 81.33, 97.95, 100.19, 100.63, 109.85, 27.49, 9.32, 68.95, 240.52, 43.06, 318.01, 304.21, 0.32, 6.68, 50.92, 10.48, 3.5, 272.21, 255.23, 334.4, 22.01, 331.29, 302.76, 344.05, 318.43], "validation": []}, {"column": "LSTAT", "alias": "LSTAT", "generated": 0, "type": "real", "max": 37.97, "min": 1.73, "mean": 12.653063241106722, "missing": 0.0, "stddev": 7.14, "outliers": [29.93, 30.81, 34.41, 29.53, 29.55, 34.77, 37.97, 30.63, 30.81, 31.99, 30.62, 30.59, 29.97, 34.37, 36.98, 34.02, 29.68], "validation": []}, {"column": "MEDV", "alias": "MEDV", "generated": 0, "type": "real", "max": 50.0, "min": 5.0, "mean": 22.532806324110677, "missing": 0.0, "stddev": 9.2, "outliers": [36.2, 35.4, 38.7, 43.8, 41.3, 50.0, 50.0, 50.0, 50.0, 37.2, 39.8, 36.2, 37.9, 50.0, 37.0, 36.4, 50.0, 42.3, 48.5, 50.0, 44.8, 50.0, 37.6, 46.7, 41.7, 48.3, 42.8, 44.0, 50.0, 36.0, 43.1, 48.8, 36.5, 50.0, 43.5, 35.2, 35.1, 45.4, 35.4, 46.0, 50.0, 37.3, 36.1, 50.0, 50.0, 50.0, 50.0, 50.0, 5.0, 6.3, 5.6, 5.0, 7.0, 7.0], "validation": []}, {"column": "Indus_Nox", "alias": "Indus_Nox", "generated": 1, "type": "real", "max": 28.348999999999997, "min": 0.882, "mean": 11.691473715415022, "missing": 0.0, "stddev": 6.95, "outliers": [], "validation": []}]}, "predictionPowerScore": [{"AGE": 1.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.1819418929, "DIS": 0.3654966571, "INDUS": 0.4743910252, "Indus_Nox": 0.5012506044, "LSTAT": 0.031921721, "MEDV": 0.12910914, "NOX": 0.501954171, "PTRATIO": 0.3398705519, "RAD": 0.160634802, "RM": 0.0, "TAX": 0.4286750281, "ZN": 0.2250916338}, {"AGE": 0.0, "B": 1.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.0, "Indus_Nox": 0.1355424654, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.1311605518, "PTRATIO": 0.0, "RAD": 0.0, "RM": 0.0, "TAX": 0.0, "ZN": 0.0}, {"AGE": 0.0, "B": 0.0, "CHAS": 1.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.0, "Indus_Nox": 0.0, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.0, "PTRATIO": 0.0, "RAD": 0.0, "RM": 0.0, "TAX": 0.0, "ZN": 0.0}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 1.0, "DIS": 0.0, "INDUS": 0.3943513349, "Indus_Nox": 0.4189261578, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.4181366721, "PTRATIO": 0.3498679351, "RAD": 0.3439931322, "RM": 0.0, "TAX": 0.3892382262, "ZN": 0.0}, {"AGE": 0.2263840804, "B": 0.0, "CHAS": 0.0, "CRIM": 0.1098016755, "DIS": 1.0, "INDUS": 0.7192599348, "Indus_Nox": 0.7875110152, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.7685256976, "PTRATIO": 0.4735583747, "RAD": 0.2010614961, "RM": 0.0, "TAX": 0.6826358869, "ZN": 0.3440131232}, {"AGE": 0.2109846408, "B": 0.007361998, "CHAS": 0.0, "CRIM": 0.4290394368, "DIS": 0.4042352387, "INDUS": 1.0, "Indus_Nox": 0.9992668546, "LSTAT": 0.0617684759, "MEDV": 0.2447842529, "NOX": 0.9094313843, "PTRATIO": 0.6640834779, "RAD": 0.4417218805, "RM": 0.0, "TAX": 0.8713858002, "ZN": 0.2662979852}, {"AGE": 0.2150085572, "B": 0.0079383689, "CHAS": 0.0, "CRIM": 0.4301136894, "DIS": 0.4073209118, "INDUS": 0.9956557648, "Indus_Nox": 1.0, "LSTAT": 0.061643455, "MEDV": 0.2450223014, "NOX": 0.9105944316, "PTRATIO": 0.6651413051, "RAD": 0.4413013992, "RM": 0.0, "TAX": 0.8682737448, "ZN": 0.2660180074}, {"AGE": 0.062603761, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0983268519, "INDUS": 0.3050010509, "Indus_Nox": 0.3759722259, "LSTAT": 1.0, "MEDV": 0.336130308, "NOX": 0.3489632496, "PTRATIO": 0.1996542817, "RAD": 0.1740047184, "RM": 0.0872401293, "TAX": 0.2588577211, "ZN": 0.130676928}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.2739922522, "Indus_Nox": 0.3660567246, "LSTAT": 0.192382389, "MEDV": 1.0, "NOX": 0.3534215648, "PTRATIO": 0.1830365575, "RAD": 0.0822970281, "RM": 0.134605752, "TAX": 0.1831393252, "ZN": 0.0969028871}, {"AGE": 0.2214231031, "B": 0.0, "CHAS": 0.0, "CRIM": 0.3529646943, "DIS": 0.3969063132, "INDUS": 0.7475713158, "Indus_Nox": 0.9853244753, "LSTAT": 0.0, "MEDV": 0.1373945132, "NOX": 1.0, "PTRATIO": 0.523485966, "RAD": 0.3203099503, "RM": 0.0, "TAX": 0.6772530451, "ZN": 0.2350807327}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0673137855, "DIS": 0.0907940989, "INDUS": 0.9172705087, "Indus_Nox": 0.9141293943, "LSTAT": 0.0, "MEDV": 0.0215388598, "NOX": 0.8518213838, "PTRATIO": 1.0, "RAD": 0.3315092837, "RM": 0.0, "TAX": 0.7655938121, "ZN": 0.1993584355}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.7525769844, "DIS": 0.0, "INDUS": 0.9783928152, "Indus_Nox": 0.9842107127, "LSTAT": 0.0, "MEDV": 0.0667400174, "NOX": 0.9687788517, "PTRATIO": 0.8051119616, "RAD": 1.0, "RM": 0.0, "TAX": 0.9212987046, "ZN": 0.0}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.0, "INDUS": 0.1465495983, "Indus_Nox": 0.1437096096, "LSTAT": 0.0, "MEDV": 0.1446528159, "NOX": 0.1310865934, "PTRATIO": 0.0976885308, "RAD": 0.005901464, "RM": 1.0, "TAX": 0.0524240323, "ZN": 0.0894627131}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.5597919875, "DIS": 0.1597987317, "INDUS": 0.96937539, "Indus_Nox": 0.9709678434, "LSTAT": 0.0, "MEDV": 0.1921251742, "NOX": 0.9474391316, "PTRATIO": 0.7269644848, "RAD": 0.6943762355, "RM": 0.0, "TAX": 1.0, "ZN": 0.075200725}, {"AGE": 0.0, "B": 0.0, "CHAS": 0.0, "CRIM": 0.0, "DIS": 0.4205587427, "INDUS": 0.8555653668, "Indus_Nox": 0.8921062367, "LSTAT": 0.0, "MEDV": 0.0, "NOX": 0.7167584404, "PTRATIO": 0.3032268596, "RAD": 0.0, "RM": 0.0, "TAX": 0.5283745907, "ZN": 1.0}], "HasBasicStats": 1, "functionsApplied": [{"functionName": "formula", "applyOn": [{"columnName": "INDUS", "type": "real", "min": "0.5", "max": "27.7", "mean": "11.1"}, {"columnName": "NOX", "type": "real", "min": "0.4", "max": "0.9", "mean": "0.6"}], "functionsData": [{"operand1": "INDUS", "operator": "add", "operand2": "NOX", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "INDUS", "type": "real", "min": "0.5", "max": "27.7", "mean": "11.1"}}, {"Columndata": {"columnName": "NOX", "type": "real", "min": "0.4", "max": "0.9", "mean": "0.6"}}, {"Columndata": ""}], "newColumnName": "Indus_Nox"}]}], "functionChanges": [{"columnName": "INDUS", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "INDUS", "operator": "add", "operand2": "NOX", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "INDUS", "type": "real", "min": "0.5", "max": "27.7", "mean": "11.1"}}, {"Columndata": {"columnName": "NOX", "type": "real", "min": "0.4", "max": "0.9", "mean": "0.6"}}, {"Columndata": ""}], "newColumnName": "Indus_Nox"}]}, {"columnName": "NOX", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "INDUS", "operator": "add", "operand2": "NOX", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "INDUS", "type": "real", "min": "0.5", "max": "27.7", "mean": "11.1"}}, {"Columndata": {"columnName": "NOX", "type": "real", "min": "0.4", "max": "0.9", "mean": "0.6"}}, {"Columndata": ""}], "newColumnName": "Indus_Nox"}]}], "fileheader": [{"field": "CRIM", "alias": "CRIM", "generated": 0, "position": 1, "type": "real"}, {"field": "ZN", "alias": "ZN", "generated": 0, "position": 2, "type": "real"}, {"field": "INDUS", "alias": "INDUS", "generated": 0, "position": 3, "type": "real"}, {"field": "CHAS", "alias": "CHAS", "generated": 0, "position": 4, "type": "real"}, {"field": "NOX", "alias": "NOX", "generated": 0, "position": 5, "type": "real"}, {"field": "RM", "alias": "RM", "generated": 0, "position": 6, "type": "real"}, {"field": "AGE", "alias": "AGE", "generated": 0, "position": 7, "type": "real"}, {"field": "DIS", "alias": "DIS", "generated": 0, "position": 8, "type": "real"}, {"field": "RAD", "alias": "RAD", "generated": 0, "position": 9, "type": "real"}, {"field": "TAX", "alias": "TAX", "generated": 0, "position": 10, "type": "real"}, {"field": "PTRATIO", "alias": "PTRATIO", "generated": 0, "position": 11, "type": "real"}, {"field": "B", "alias": "B", "generated": 0, "position": 12, "type": "real"}, {"field": "LSTAT", "alias": "LSTAT", "generated": 0, "position": 13, "type": "real"}, {"field": "MEDV", "alias": "MEDV", "generated": 0, "position": 14, "type": "real"}, {"field": "Indus_Nox", "alias": "Indus_Nox", "generated": 1, "position": 15, "type": "real"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run 3110sqlsinkHooks.ipynb
try:
	#sinkPreExecutionHook()

	sqlserver = RDBMSConnector.put(spark, "{'host': '3.17.87.4', 'port': '1433', 'password': 'Passw0rd@123', 'database': 'testdb', 'user': 'SA', 'optionalDB': '', 'table_option': 'create_new_table', 'table': 'BostonNew', 'new_table_name': 'BostonNew', 'writemode': '', 'is_header': 'Use Header Line', 'use_generatedColumns': False, 'generatedColumns': [], 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sinkPostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)
